In [1]:
import numpy as np
import pandas as pd
import re
from tqdm.notebook import tqdm
import pickle
import seaborn as sns
import sys

sys.path.insert(0, "../")
from ope_estimators import *

from plotnine import *

%load_ext autoreload
%autoreload 2

from siuba import spread, gather, mutate, filter, select, left_join, case_when, _

### Load data and make plots

In [2]:
with open("yahoo_results_final.pkl", "rb") as f:
    plot_df = pickle.load(f)

In [3]:
plot_df

,L,variable,value,method,type,T
0,0.200000,imputation,0.050517,imputation,point_estim,0.25
1,0.216327,imputation,0.050517,imputation,point_estim,0.25
2,0.232653,imputation,0.050517,imputation,point_estim,0.25
3,0.248980,imputation,0.050517,imputation,point_estim,0.25
4,0.265306,imputation,0.050517,imputation,point_estim,0.25
...,...,...,...,...,...,...
507,11.000000,manski_lower,0.049772,manski,partial_id,0.50
508,12.000000,manski_lower,0.049772,manski,partial_id,0.50
509,13.000000,manski_lower,0.049772,manski,partial_id,0.50
510,14.000000,manski_lower,0.049772,manski,partial_id,0.50


In [3]:
ribbon_df_psi = (plot_df 
 >> filter(_.method == "lip")
 >> spread("variable", "value"))

ribbon_df_unbiased = (plot_df
                      >> filter(_.method == "unbiased")
                      >> spread("variable", "value"))


plot_df_full = left_join(plot_df, ribbon_df_psi, on = ["L", "method", "type", "T"])
plot_df_full = left_join(plot_df_full, ribbon_df_unbiased, on = ["L", "method", "type", "T"])

plot_df_full["T_label"] = plot_df_full.apply(lambda x : "T="+str(x["T"]), axis =1)

plot_df_full.head()

,L,variable,value,method,type,T,psi_minus,psi_plus,ci_lower,ci_upper,unbiased,T_label
0,0.200000,imputation,0.050517,imputation,point_estim,0.25,NaN,NaN,NaN,NaN,NaN,T=0.25
1,0.216327,imputation,0.050517,imputation,point_estim,0.25,NaN,NaN,NaN,NaN,NaN,T=0.25
2,0.232653,imputation,0.050517,imputation,point_estim,0.25,NaN,NaN,NaN,NaN,NaN,T=0.25
3,0.248980,imputation,0.050517,imputation,point_estim,0.25,NaN,NaN,NaN,NaN,NaN,T=0.25
4,0.265306,imputation,0.050517,imputation,point_estim,0.25,NaN,NaN,NaN,NaN,NaN,T=0.25


In [5]:
#Full plot for appendix

p = (ggplot(plot_df_full, aes(x="L", group="variable", color="method", linetype = "type"))
         + geom_line(aes(y="value"))
         + geom_ribbon(aes(ymin = "psi_minus", ymax = "psi_plus"), fill = "black", alpha = 0.1, show_legend = False)
         + geom_ribbon(aes(ymin = "ci_lower", ymax = "ci_upper"), fill = "orange", alpha = 0.1, show_legend = False)
         + labs(x="Lipschitz constant, L", y="Off-policy value",
                color="Method", linetype="Quantity")
         + theme_bw()
         + scale_color_manual(values={"imputation": "blue", "lip": "black",
                               "unbiased": "orange", "manski": "grey"},
                               labels = ["Imputation", "Lipschitz", "Manski", "Infeasible sample"])
         + scale_linetype_manual(values = {"ci":"dotted", "partial_id":"solid", "point_estim":"dashed"},
                                   labels = ["CI", "Partial id.", "Point estimate"])
         + theme(panel_grid_minor = element_blank()) 
         + facet_wrap("T_label"))

# p

ggsave(p, "yahoo_lipschitz_manski_full.pdf", width = 6, height = 3)
# ggsave(p, "test.pdf", width = 6, height = 3)


/Users/samirkhan/opt/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:718: PlotnineWarning: Saving 6 x 3 in image.
/Users/samirkhan/opt/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Filename: yahoo_lipschitz_manski_full.pdf


In [6]:
#Partial plot for main text

plot_df_partial = plot_df_full.query("T in [0.25, 0.3, 0.35, 0.4]")
plot_df_partial = plot_df_partial >> filter(_.L <= 1)

p = (ggplot(plot_df_partial, aes(x="L", group="variable", color="method", linetype = "type"))
         + geom_line(aes(y="value"))
         + geom_ribbon(aes(ymin = "psi_minus", ymax = "psi_plus"), fill = "black", alpha = 0.1, show_legend = False)
         + geom_ribbon(aes(ymin = "ci_lower", ymax = "ci_upper"), fill = "orange", alpha = 0.1, show_legend = False)
         + labs(x="Lipschitz constant, L", y="Off-policy value",
                color="Method", linetype="Quantity")
         + theme_bw()
         + scale_color_manual(values={"imputation": "blue", "lip": "black",
                               "unbiased": "orange", "manski": "grey"},
                               labels = ["Imputation", "Lipschitz", "Manski", "Infeasible sample"])
         + scale_linetype_manual(values = {"ci":"dotted", "partial_id":"solid", "point_estim":"dashed"},
                                   labels = ["CI", "Partial id.", "Point estimate"])
         + theme(panel_grid_minor = element_blank()) 
         + facet_wrap("T_label", nrow = 1))

p

ggsave(p, "yahoo_lipschitz_manski.pdf", width = 6, height = 3)





/Users/samirkhan/opt/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:718: PlotnineWarning: Saving 6 x 3 in image.
/Users/samirkhan/opt/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Filename: yahoo_lipschitz_manski.pdf


### Compare widths of bounds

In [76]:
(plot_df >> 
 filter(_.L == 1, _.type == "partial_id") >> 
 spread("variable", "value") >> 
 mutate(manski_width = _.manski_upper - _.manski_lower,
        lip_width = _.psi_plus - _.psi_minus,
        width = case_when(_, {np.isnan(_.manski_width):_.lip_width, 
                              True:_.manski_width})) >> 
 select(_.method, _.T, _.width) >> 
 spread("method", "width") >> 
 mutate(pct_reduction = 100*(_.manski - _.lip)/_.manski))   

,T,lip,manski,pct_reduction
0,0.25,0.022213,0.083978,73.549161
1,0.30,0.011258,0.053787,79.069672
2,0.35,0.005675,0.034801,83.692755
3,0.40,0.002688,0.021515,87.505120
4,0.45,0.000899,0.010559,91.490057
5,0.50,0.000000,0.000000,NaN


### Count % overlap violations

In [77]:
ts = "1241180700"
with open(f"dfs/ts_{ts}_date_20090501_clicks.pkl", "rb") as f:
    df = pickle.load(f)

with open(f"dfs/ts_{ts}_date_20090501_articles.pkl", "rb") as f:
    article_df = pickle.load(f)

article_df.columns = ["V0", "V1", "V2", "V3", "V4", "V5", "A", "click_rate"]
article_df.head()

pool = df["pool"][0]

threshold = np.median(article_df["V0"])

subpool = article_df.query(f"V0 > {threshold}")["A"].values

ope_df = pd.merge(df, article_df.filter(["A", "V0"]), on="A")
ope_df["in_subpool"] = ope_df["V0"] > threshold

ope_df = ope_df.filter(["A", "Y", "X0", "X1", "X2", "X3", "X4", "in_subpool"])
ope_df = ope_df.query("in_subpool or X3 > 0.5")
ope_df.head()


,A,Y,X0,X1,X2,X3,X4,in_subpool
0,109509,0,0.020620,0.185252,0.006546,0.787500,0.000083,True
1,109509,0,0.007116,0.000410,0.001595,0.009283,0.981597,True
2,109509,0,0.052519,0.336099,0.000449,0.120247,0.490686,True
3,109509,0,0.022999,0.000328,0.372671,0.602106,0.001896,True
4,109509,0,0.605156,0.077316,0.029215,0.270011,0.018302,True


In [78]:
print(f"{len(df)} total points")
print(f"{len(ope_df)} subsampled points")

16628 total points
10086 subsampled points


In [79]:
def overlap_violations(t):
    return len(ope_df.query(f"X3 < 0.5 and X3 > {t}"))

for t in [0.25, 0.3, 0.35, 0.4]:
    print(f"t={t} has {round(100*overlap_violations(t)/len(ope_df), 2)}% overlap violations")

t=0.25 has 16.8% overlap violations
t=0.3 has 10.76% overlap violations
t=0.35 has 6.96% overlap violations
t=0.4 has 4.3% overlap violations
